In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

In [2]:
from transformers import pipeline

In [3]:
finetuned_model = 'finetuned-ai-genessay-detect-distilbert_ext_train2'
BERT_MODEL = 'distilbert_base_cased'
pipe = pipeline("text-classification", finetuned_model, tokenizer=BERT_MODEL)

In [4]:
train = pd.read_csv('train_essays.csv')
test = pd.read_csv('test_essays.csv')

In [5]:
for i in tqdm(range(50)): #train.shape[0]
    text = train.loc[i, 'text']
#     print(len(text))
    cum_score = []
    for j in range(int(len(text)/1000)+1):
#         print(int(len(text)/1000)+1)
        sample_text = text[j*1000:(j+1)*1000]
        out = pipe(sample_text)[0]
        if out['label'] == 'Not AI-generated':
            score = 1 - out['score']
        else:
            score = out['score']
        cum_score.append(score)

#     if np.average(cum_score)>0.50 and any([x<0.45 for x in cum_score]):
#         print(i, np.average(cum_score), cum_score)

100%|██████████| 50/50 [00:36<00:00,  1.36it/s]


In [6]:
preds_test = []
for i in tqdm(range(test.shape[0])): #test.shape[0]
    text = test.loc[i, 'text']
    cum_score = []
    for j in range(int(len(text)/1000)+1):
        sample_text = text[j*1000:(j+1)*1000]
        out = pipe(sample_text)[0]
        if out['label'] == 'Human':
            score = 1 - out['score']
        else:
            score = out['score']
        cum_score.append(score)
        
    preds_test.append(np.average(cum_score))

100%|██████████| 3/3 [00:00<00:00, 34.31it/s]


In [7]:
from sklearn import preprocessing
import numpy as np

x_array = np.array(preds_test)

normalized_arr = preprocessing.normalize([x_array])
norm_preds_test = normalized_arr[0]

In [8]:
pd.DataFrame({'id':test["id"],'generated':norm_preds_test}).to_csv('submission.csv', index=False)